# 不等式のQUBO変換を用いて基底エネルギー以下のサンプルを得る
- small data
- SA using Pyqubo

### import

In [1]:
#!pip3 install pyqubo

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression
import random
from pyqubo import Array, OneHotEncInteger, solve_qubo
import scipy.stats
import statistics

### 各変数

In [3]:
# X
matrix = np.random.randn(3,5)
X_ori= pd.DataFrame(matrix, columns=list('ABCDE'))
X_ori

# CSV ファイル (df1.csv) として出力
#X_ori.to_csv("../input/df1.csv")
X_ori = pd.read_csv("../input/df1.csv", sep=',', index_col=0)
X_ori

,A,B,C,D,E
0,-1.297303,-0.049925,0.185726,-0.580797,0.926729
1,1.129216,0.263337,-2.072408,0.269691,-0.565672
2,-1.140225,0.447902,-0.641875,-0.032109,0.415254
3,-1.004241,0.700738,-1.250194,-1.827180,1.972915
4,1.119464,0.248735,1.328148,0.295280,1.007603


In [4]:
# y
ori_y = pd.Series([1, 0, 1, 1, 0])
ori_y

0    1
1    0
2    1
3    1
4    0
dtype: int64

In [5]:
selected_col_num = 2

## 特徴量選択
 - select_cols, else_colsを決める

In [6]:
from decimal import *

In [7]:
#標準偏差が0である列をなくす
for each_col in X_ori.columns:
    if not X_ori[each_col].std()>0:
        X_ori.drop(each_col, axis=1)

In [8]:
#Xの要素を標準化(Standardization : 平均0分散1)
X_standar = pd.DataFrame(scipy.stats.zscore(X_ori), columns=X_ori.columns)
X_standar

,A,B,C,D,E
0,-0.948036,-1.502995,0.576689,-0.259926,0.211537
1,1.224875,-0.237599,-1.350141,0.814379,-1.588719
2,-0.807376,0.507934,-0.129490,0.433156,-0.405446
3,-0.685604,1.529245,-0.648559,-1.834311,1.473533
4,1.216142,-0.296585,1.551500,0.846703,0.309095


In [9]:
selector = SelectKBest(score_func=f_regression, k=selected_col_num) 
selector.fit(X_standar, ori_y)
mask = selector.get_support() 

In [10]:
select_cols = []
else_cols = []
for ii in range(len(mask)):
    if mask[ii] == 0:
        else_cols.append(X_standar.columns[ii])
    else:
         select_cols.append(X_standar.columns[ii])

select_cols

['A', 'D']

## すべての不等式が成り立つ：条件
したいこと→ハミルトニアンを最小化させる（xを最適化）

In [11]:
max_X_element = X_standar.max(axis=1).max() 
min_X_element = X_standar.min(axis=1).min() 

num_samples = X_standar.shape[0]
num_cols = X_standar.shape[1]

In [12]:
# W : ay - byの最大値

#xの要素で最大値とxの要素の最小値を知れば推定はできる
W = int((max_X_element *1*num_samples - min_X_element*1*num_samples)//1)

In [13]:
y = Array.create('y', shape=num_samples, vartype='BINARY')

In [14]:
#内積ではなくて相関係数（もどき）を使うことにする
m_list = []
for i in range(selected_col_num):
    for j in range(num_cols - selected_col_num):
        sel_list = X_standar[select_cols[i]].values.tolist()
        else_list = X_standar[else_cols[j]].values.tolist()
        each_m = y.dot(Array(sel_list))/statistics.pstdev(sel_list)**2**0.5 - y.dot(Array(else_list))/statistics.pstdev(else_list)**2**0.5
        m_list.append(each_m)

In [15]:
m_list

[((((Binary(y[0])*Num(-0.9480364220048371))+(Binary(y[1])*Num(1.2248745312842604))+(Binary(y[2])*Num(-0.807375760737013))+(Binary(y[3])*Num(-0.6856040906829342))+(Binary(y[4])*Num(1.2161417421405236)))*Num(1.0))+((((Binary(y[0])*Num(-1.5029954355850952))+(Binary(y[1])*Num(-0.23759877688955197))+(Binary(y[2])*Num(0.5079341001366204))+(Binary(y[3])*Num(1.5292454453138022))+(Binary(y[4])*Num(-0.2965853329757752)))*Num(1.0))*Num(-1))),
 ((((Binary(y[0])*Num(-0.9480364220048371))+(Binary(y[1])*Num(1.2248745312842604))+(Binary(y[2])*Num(-0.807375760737013))+(Binary(y[3])*Num(-0.6856040906829342))+(Binary(y[4])*Num(1.2161417421405236)))*Num(1.0))+((((Binary(y[0])*Num(0.5766893301292317))+(Binary(y[1])*Num(-1.3501409788166288))+(Binary(y[2])*Num(-0.12948960878414023))+(Binary(y[3])*Num(-0.6485588567394989))+(Binary(y[4])*Num(1.5515001142110358)))*Num(1.0000000000000002))*Num(-1))),
 ((((Binary(y[0])*Num(-0.9480364220048371))+(Binary(y[1])*Num(1.2248745312842604))+(Binary(y[2])*Num(-0.807375760

In [16]:
H_sum = sum((OneHotEncInteger("x", 1, W, strength = 1)-m)**2 for m in m_list)
model = H_sum.compile()

In [17]:
qubo, offset = model.to_qubo()

#PyQUBOによるSA
raw_solution = solve_qubo(qubo)

# 得られた結果をデコードする
decoded_solution, broken, energy = model.decode_solution(raw_solution, vartype="BINARY")
decoded_solution["y"] 

{0: 0, 1: 1, 2: 0, 3: 0, 4: 0}

### 出力結果の記録
|  y  | energy |全探索で得られたかどうか|コメント|
| ---- | ---- | ---- | ---- |
| (1, 1, 1, 0, 1) |8.15741678762907||コメント|
| (0, 1, 0, 0, 1) |16.121628397329548|✔|全探索ではエネルギー0だったのに|
| (1, 1, 0, 0, 0) |1.5653947899485452||コメント|
| (0, 1, 1, 0, 1) |6.302600000842226||コメント|
| (0, 1, 0, 0, 0) |2.3698131291032265||コメント|
| (1, 1, 1, 1, ,1) |6.0||内積ならいいが相関係数ではエラーとなる|
| (0, 1, 0, 0, 1) |15.059246135573005|✔|全探索ではエネルギー0だったのに、しかもエネルギーが前回のpyqubo出力値から変化した|
| (0, 1, 0, 0, 0) |2.3698131291032265||コメント|
| (0, 1, 1, 0, 0) |6.729759524817439||コメント|
| (1, 1, 0, 0, 0) |1.5653947899485452||コメント|
| (0, 1, 1, 1, 1) |8.056104055714982||コメント|
| (0, 1, 1, 1, 1) |8.056104055714982||コメント|
| (0, 1, 0, 1, 1) |8.916665170674364||コメント|
| (1, 1, 0, 0, 0) |11.012739511827256||エネルギーが前回のpyqubo出力値から変化した|
| (0, 0, 0, 0, 0) |6.0||内積ならいいが相関係数ではエラーとなる|
| (0, 1, 0, 1, 1) |8.916665170674364||コメント|
| (0, 1, 1, 0, 0) |6.729759524817439||コメント|
| (0, 1, 1, 0, 1) |6.302600000842226||コメント|
| (1, 1, 1, 1, ,1) |18.0||内積ならいいが相関係数ではエラーとなる, しかもエネルギーが前回のpyqubo出力値から変化した|
:
:
:

In [18]:
energy 

2.3698131291032265

#### 課題
元のyで求められるエネルギーは0であるので、適した解以外も多く出力されていることになる
<br/>

### PyQuboで出力されたyを使ってエネルギーを求めてみる

In [243]:
def make_ax_by(y_after):
    ij = []
    ax_by = []
    for i in range(selected_col_num):
        for j in range(num_cols - selected_col_num):
            row_each_axby = abs(np.corrcoef(X_standardization[select_cols[i]], y_after)[0, 1]) - abs(np.corrcoef(X_standardization[else_cols[j]], y_after)[0, 1])
            ax_by.append(int(row_each_axby + 1))
            ij.append((i, j))
    return ij, ax_by

In [244]:
def H(ax_by, index_ax_by):
    z = [0]*W
        
    each_ax_by = ax_by[index_ax_by]
    int_each_axby = each_ax_by //1
    sum_w_z = 0
    
    if int_each_axby >= 1:
        z[int(int_each_axby)] = 1
        sum_w_z = sum([int_each_axby*z[w] for w in range(W)])
    else:
        z[0] = 1
        sum_w_z = sum([1*z[w] for w in range(W)])
        
    sum_z = sum(z)

    ans_h =  (1-sum_z)**2 + (sum_w_z -  int(int_each_axby))**2
    return ans_h

In [245]:
def H_sum(ax_by):
        H_sum =  sum([H(ax_by, index_ax_by) for index_ax_by in range(len(ax_by))])
        return H_sum

In [246]:
y = pd.Series([0, 1, 1, 0, 1])
ij, ax_by = make_ax_by(y)
pyqubo_H_sum = H_sum(ax_by)
pyqubo_H_sum

1

In [22]:
if ori_H_sum >= energy:
    print('SAは成功していると考えられる')

SAは成功していると考えられる


## SAで出したyが全ての不等式を満たしているかどうか

In [23]:
sol_y =  pd.Series([0, 1, 0])

In [24]:
sol_ax_by = make_ax_by(sol_y)
sol_ax_by 

[2, 2, 2, 0]

In [25]:
ok = 0
for  ii in sol_ax_by :
    if ii >= 0:
        ok += 1
if ok == len(sol_ax_by):
    print('SAで出したyは全ての不等式を満たしている')

SAで出したyは全ての不等式を満たしている


## 結果
- `SAは成功していると考えられる`, `SAで出したyは全ての不等式を満たしている`両方を満たしていない結果も出る